<a href="https://colab.research.google.com/github/LorenzoTinfena/BestSpiderWeb/blob/master/BestSpiderWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BestSpiderWeb
## Problem
A city without roads has a grain producer, an egg producer and a hotel.
The mayor also wants to build a pasta producer and a restaurant in the future. He also wants to build roads like in the picture, so that the producer can easily take the wheat and eggs to make pasta, and the restaurant can easily buy pasta, welcome hotel people, and buy eggs for other preparations.

<img src="https://github.com/LorenzoTinfena/BestSpiderWeb/blob/master/assets/city0.png?raw=1" width="300"/>

**Goal:** to build roads costs, you have to make them as short as possible.

<img src="https://github.com/LorenzoTinfena/BestSpiderWeb/blob/master/assets/city1.png?raw=1" width="400"/>


---


**In other words:** In an Euclidean space there is a graph with constant edges and with 2 types of nodes, one with constant coordinates, the other with a variable coordinates.

**Goal:** To find the positions of the variable nodes in order to have the smaller sum of the length of the edges


---


## Solution
$$
N_0[c] = \sum_{i \in N}\sum_{v \in P_{N_0  \longleftrightarrow i}}\frac{\sum O_i[c]}{v}
$$
where
*   $$N_0$$

*   $$c$$
coordinates
*   $$N$$
set of nodes with variable coordinates reachable from N with 0 passing only through nodes belonging to N
*   $$O$$
set of nodes with constant coordinates
*   $$O_i$$
set of nodes belonging to "O" adjacent to "i"
*   $$P_{N_0  \rightarrow i}$$
set of all possible paths (infinite for lenght of "N" greater than 1") between node "N with 0" to node "i",
passing only through nodes belonging to N
*   $$v$$
Or path, is a multiplication of the number of edges for all the nodes it crosses, "N with 0" included, "i" included,
(e.g. if it starts from a node that has 7 adjacent edges, then goes through one that has 2,
and ends up with one having 3, the calculation will be 7 * 2 * 3 = 42

In [4]:
import threading as th
import numpy as np
import ipywidgets as widgets
from IPython.display import display


class Node:
    NoCoordinates = 2  # @param {type:"integer"}

    def __init__(self):
        self.AdjacentNodes = []
        self.Constant = False

    def __init__(self, coordinates: np.ndarray):
        if len(coordinates) != Node.NoCoordinates:
            raise Exception('wrong number of coordinates')
        self.AdjacentNodes = []
        self.Coordinates = coordinates
        self.Constant = True

    def AddAdjacentNode(self, item: 'Node'):
        self.AdjacentNodes += item


    class VirtualNode:
        def __init__(self, nodeBase: 'Node'):
            self.ActualNode = nodeBase
            self.SumConstantNodes = np.zeros(Node.NoCoordinates)
            for item in nodeBase.AdjacentNodes:
                if item.Constant:
                    self.SumConstantNodes += item.Coordinates
            self.NumTmpPath = len(nodeBase.AdjacentNodes)

        def Copy(self, moltiplicator: int) -> 'VirtualNode':
            item = Node.VirtualNode()
            item.ActualNode = self.ActualNode
            item.SumConstantNodes = self.SumConstantNodes
            item.NumTmpPath = self.NumTmpPath * moltiplicator
            return item





def ComputeBestSpiderWeb(variablesNodes: list):
    # initialize coordinates of variables nodes
    for item in variablesNodes:
        item.Coordinates = np.zeros(Node.NoCoordinates)

    virtualNodes = []
    for item in variablesNodes:
        virtualNodes += Node.VirtualNode(item)

    th.Thread(target=_WeaveTheSpiderWeb, args=(virtualNodes)).start()


def _WeaveTheSpiderWeb(virtualNodes: list):
    global THREADSTOPPED
    while THREADSTOPPED:
        nextVirtualNodes = []
        for item in virtualNodes:
            item.ActualNode.Coordinates += item.SumConstantNodes / item.NumTmpPath
            for AdjacentItem in item.ActualNode.AdjacentNodes:
                if THREADSTOPPED:
                    return
                nextVirtualNodes += item.Copy(moltiplicator=len(AdjacentItem.AdjacentNodes))
        virtualNodes = nextVirtualNodes

In [5]:
def _OnButtonClicked(b):
    global THREADSTOPPED
    THREADSTOPPED = True

variablesNodes = []
constantNodes = []


ComputeBestSpiderWeb(variablesNodes)
BUTTON = widgets.Button(description="Stop")
OUTPUT = widgets.Output()
BUTTON.on_click(_OnButtonClicked, remove=True)
display(BUTTON, OUTPUT)
THREADSTOPPED: bool = False

Button(description='Stop', style=ButtonStyle())

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: _WeaveTheSpiderWeb() missing 1 required positional argument: 'virtualNodes'



Output()